## 1. Import libraries

In [ ]:
import xgboost as xgb
print(xgb.__version__)
import numpy as np
import pandas as pd
import optuna
import mlflow
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import mlflow.xgboost

## 2. Load processed datasets

In [ ]:
train_df = pd.read_csv("/Users/leduongkhoa/Regression_MachineLearning_End2End/data/processed/feature_engineered_train.csv")
eval_df = pd.read_csv("/Users/leduongkhoa/Regression_MachineLearning_End2End/data/processed/feature_engineered_eval.csv")

In [ ]:
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape: ", X_train.shape, y_train.shape)
print("Eval shape: ", X_eval.shape, y_eval.shape)

## 3. Define Optuna objective function and MLflow

In [ ]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist"
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # log parameters and metrics to MLflow
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

## 4. Run Optuna with MLflow

In [ ]:
mlflow.set_tracking_uri("/Users/leduongkhoa/Regression_MachineLearning_End2End/mlruns")
mlflow.set_experiment("XGBoost_Hyperparameter_Tuning")

study = optuna.create_study(direction="minimize", study_name="XGBoost_Hyperparameter_Tuning")
study.optimize(objective, n_trials=15)

print(f"Best params: {study.best_trial.params}")

## 5. Train the model with the best params and log into MLflow

In [ ]:
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
mae = float(mean_absolute_error(y_eval, y_pred))
r2 = float(r2_score(y_eval, y_pred))

print(f"Best Model Performance on Eval Set:\nRMSE: {rmse}\nMAE: {mae}\nR2: {r2}")

with mlflow.start_run(run_name="Best_XGBoost_Model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, artifact_path="model")